In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler
from datetime import datetime
from sklearn.decomposition import PCA,KernelPCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph
import seaborn as sns
color = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_rows',100)
prods=pd.read_csv('/Users/shashank/Desktop/Clustering/cproducts.csv')
tender=pd.read_csv('/Users/shashank/Desktop/Clustering/ctender.csv')

In [ ]:
tender.isnull().sum()>0

In [ ]:
# To fill missing states from pincode.
dic={11:'Delhi',12:'Haryana',13:'Haryana',14:'Punjab',15:'Punjab',16:'Punjab',17:'Himachal Pradesh',
    18:'Jammu & Kashmir',19:'Jammu & Kashmir',20:'Uttar Pradesh',21:'Uttar Pradesh',22:'Uttar Pradesh',
    23:'Uttar Pradesh',24:'Uttar Pradesh',25:'Uttar Pradesh',26:'Uttar Pradesh',27:'Uttar Pradesh',28:'Uttar Pradesh',
    30:'Rajasthan',31:'Rajasthan',32:'Rajasthan',33:'Rajasthan',34:'Rajasthan',36:'Gujarat',37:'Gujarat',38:'Gujarat',
    39:'Gujarat',40:'Maharashtra',41:'Maharashtra',42:'Maharashtra',43:'Maharashtra',44:'Maharashtra',45:'Madhya Pradesh',
    46:'Madhya Pradesh',47:'Madhya Pradesh',48:'Madhya Pradesh',49:'Madhya Pradesh',50:'Andhra Pradesh',51:'Andhra Pradesh',
    52:'Andhra Pradesh',53:'Andhra Pradesh',56:'Karnataka',57:'Karnataka',58:'Karnataka',59:'Karnataka',
    60:'Tamil Nadu',61:'Tamil Nadu',62:'Tamil Nadu',63:'Tamil Nadu',64:'Tamil Nadu',67:'Kerala',68:'Kerala',69:'Kerala',
    70:'West Bengal',71:'West Bengal',72:'West Bengal',73:'West Bengal',74:'West Bengal',75:'Orissa',76:'Orissa',77:'Orissa',
    78:'Assam',79:'North Eastern',80:'Bihar',81:'Bihar',82:'Bihar',83:'Jharkand',84:'Bihar',85:'Bihar',90:'Army',91:'Army',
    92:'Army',93:'Army',94:'Army',95:'Army',96:'Army',97:'Army',98:'Army',99:'Army',35:'Madhya Pradesh',1:'Delhi',88:'Jharkhand',
    55:'Karnataka',66:'Karnataka',54:"Karnataka",65:'Tamil Nadu',10:'Tamil Nadu'}

# To fill missing states from store location when pincode is also missing.
dic1={'BB-MADURAI-KOCHADAI VILLAGE':'Tamil Nadu','BB-INDORE-TREASURE ISLAND':"Madhya Pradesh",
     'BB-INDORE-MALHAR MEGA MALL':'Madhya Pradesh','BB-LUDHIANA-FEROZEPUR-MF':'Punjab','BB-JAMSHEDPUR-NH-33':'Jharkand',
     'BB-HUBLI-GOKUL ROAD-AKSHAY CEN':'Kanataka','BB-AMRITSAR-TRILIUM MALL':'Punjab'}

In [ ]:
#Fill states features
zips=(prods['PinCode'].unique())
zips=zips[~np.isnan(zips)]

prods['State']=prods['PinCode'].apply(lambda x: dic[int(x)/10000] if x in zips else x)
tender['State']=tender['PinCode'].apply(lambda x: dic[int(x)/10000] if x in zips else x)

states=list(prods['State'].unique())
states.remove(np.nan)

prods['State']=prods.apply(lambda x: dic1[x['store_description']] if x['State'] not in states else x['State'] ,axis=1)
tender['State']=tender.apply(lambda x: dic1[x['store_description']] if x['State'] not in states else x['State'] ,axis=1)

# Encode states
le=LabelEncoder()
prods["State"]=le.fit_transform(prods["State"])
tender["State"]=le.fit_transform(tender["State"])

prods.drop(["PinCode"],axis=1,inplace=True)
tender.drop(["PinCode"],axis=1,inplace=True)

In [ ]:
# Fill missing gender with no_gender and encode.

prods['Gender'].fillna('no_gender', inplace=True)
tender['Gender'].fillna('no_gender', inplace=True)
le=LabelEncoder()
prods["Gender"]=le.fit_transform(prods["Gender"])
tender["Gender"]=le.fit_transform(tender["Gender"])

In [ ]:
# Fill missing DOB with the most relevant date and convert DOB into age

prods['DOB']=prods['DOB'].fillna('1983-1-1')
prods['DOB']=prods['DOB'].replace(['NANA'],['1983-1-1'])
prods['DOB']=prods['DOB'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d').year)
prods["DOB"]=2017-prods["DOB"]

tender['DOB']=tender['DOB'].fillna('1983-1-1')
tender['DOB']=tender['DOB'].replace(['NANA'],['1983-1-1'])
tender['DOB']=tender['DOB'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d').year)
tender["DOB"]=2017-tender["DOB"]

In [ ]:
prods['transactionDate']=prods['transactionDate'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
prods['Year']=prods['transactionDate'].apply(lambda x:x.year)
prods['Month']=prods['transactionDate'].apply(lambda x:x.month)
prods['Weekday']=prods['transactionDate'].apply(lambda x:x.weekday())
prods['Day']=prods['transactionDate'].apply(lambda x:x.day)

In [ ]:
tender['transactionDate']=tender['transactionDate'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
tender['Year']=tender['transactionDate'].apply(lambda x:x.year)
tender['Month']=tender['transactionDate'].apply(lambda x:x.month)
tender['Weekday']=tender['transactionDate'].apply(lambda x:x.weekday())
tender['Day']=tender['transactionDate'].apply(lambda x:x.day)
tender["date"]=(tender["Year"]*100)+tender["Month"]
tender["freq"]=(tender["date"]*100)+tender["Day"]

In [ ]:
le=LabelEncoder()
prods["discountUsed"]=le.fit_transform(prods["discountUsed"])

In [ ]:
le=LabelEncoder()
prods["promo_code"]=le.fit_transform(prods["promo_code"])
prods.drop(['promotion_description'],axis=1,inplace=True)

In [ ]:
tender.drop(['transactionDate'],axis=1,inplace=True)
prods.drop(['transactionDate'],axis=1,inplace=True)

In [ ]:
tender.drop(['store_description'],axis=1,inplace=True)
prods.drop(['store_description'],axis=1,inplace=True)

In [ ]:
tender["store_code"].unique()

In [ ]:
prods["product_code"]=prods["product_code"].apply(lambda x: int(str(x)[:6]))
prods.drop(['product_description'],axis=1,inplace=True)

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==4796]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 6307:
        val = 1
    elif row["payment_amount_by_tender"] > 6307 and row["payment_amount_by_tender"] <= 13213:
        val = 2
    elif row["payment_amount_by_tender"] > 13213 and row["payment_amount_by_tender"] <= 22664:
        val = 3
    elif row["payment_amount_by_tender"] > 22664 and row["payment_amount_by_tender"] <= 43330:
        val = 4
    else:
        val = 5
    return val

Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df7=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==4843]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 6147:
        val = 1
    elif row["payment_amount_by_tender"] > 6147 and row["payment_amount_by_tender"] <= 12661:
        val = 2
    elif row["payment_amount_by_tender"] > 12661 and row["payment_amount_by_tender"] <= 21977:
        val = 3
    elif row["payment_amount_by_tender"] > 21977 and row["payment_amount_by_tender"] <= 41680:
        val = 4
    else:
        val = 5
    return val

Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df6=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==3692]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 4335:
        val = 1
    elif row["payment_amount_by_tender"] > 4335 and row["payment_amount_by_tender"] <= 8565:
        val = 2
    elif row["payment_amount_by_tender"] > 8565 and row["payment_amount_by_tender"] <= 13255:
        val = 3
    elif row["payment_amount_by_tender"] > 13255 and row["payment_amount_by_tender"] <= 23122:
        val = 4
    else:
        val = 5
    return val

Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df5=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==2615]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 5839:
        val = 1
    elif row["payment_amount_by_tender"] > 5839 and row["payment_amount_by_tender"] <= 11538:
        val = 2
    elif row["payment_amount_by_tender"] > 11538 and row["payment_amount_by_tender"] <= 17561:
        val = 3
    elif row["payment_amount_by_tender"] > 17561 and row["payment_amount_by_tender"] <= 29654:
        val = 4
    else:
        val = 5
    return val

Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df4=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==2655]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 3256:
        val = 1
    elif row["payment_amount_by_tender"] > 3256 and row["payment_amount_by_tender"] <= 7089:
        val = 2
    elif row["payment_amount_by_tender"] > 7089 and row["payment_amount_by_tender"] <= 12941:
        val = 3
    elif row["payment_amount_by_tender"] > 12941 and row["payment_amount_by_tender"] <= 23289:
        val = 4
    else:
        val = 5
    return val

Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df3=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==2906]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 1412:
        val = 1
    elif row["payment_amount_by_tender"] > 1412 and row["payment_amount_by_tender"] <= 3493:
        val = 2
    elif row["payment_amount_by_tender"] > 3493 and row["payment_amount_by_tender"] <= 6600:
        val = 3
    elif row["payment_amount_by_tender"] > 6600 and row["payment_amount_by_tender"] <= 13396:
        val = 4
    else:
        val = 5
    return val

Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df2=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:
# Extract freq,recency and monetary features for each store 

data=tender[tender["store_code"]==4986]

#freq
Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 3:
        val = 1
    elif row["date"] >=4 and row["date"] <= 8:
        val = 2
    elif row["date"] > 8 and row["date"] <= 11:
        val = 3
    else :
        val = 4
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)

# plt.figure(figsize=(12,8))
# sns.countplot(x="Freq_Flag", data=Cust_freq_count, color=color[1])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Freq_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Freq_Flag", fontsize=15)
# plt.show()

#recency
def f(row):
    if row["date"] > 201701:
        val = 5
    elif row["date"] <= 201701 and row["date"] > 201608:
        val = 4
    elif row["date"] <= 201608 and row["date"] > 201603:
        val = 3
    elif row["date"] <= 201603 and row["date"] > 201510:
        val = 2
    else:
        val = 1
    return val

data["Recency_Flag"] = data.apply(f, axis=1)
data_freq = data.groupby("customerID",as_index=False)["Recency_Flag"].max()
data=pd.merge(data,data_freq,on=["customerID"],how="left")

# plt.figure(figsize=(12,8))
# sns.countplot(x="Recency_Flag", data=data_freq, color=color[3])
# plt.ylabel("Count", fontsize=12)
# plt.xlabel("Recency_Flag", fontsize=12)
# plt.xticks(rotation="vertical")
# plt.title("Frequency of Recency_Flag", fontsize=15)
# plt.show()


Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()

def f(row):
    if row["payment_amount_by_tender"] <= 5937:
        val = 1
    elif row["payment_amount_by_tender"] > 5937 and row["payment_amount_by_tender"] <= 12718:
        val = 2
    elif row["payment_amount_by_tender"] > 12718 and row["payment_amount_by_tender"] <= 21318:
        val = 3
    elif row["payment_amount_by_tender"] > 21318 and row["payment_amount_by_tender"] <= 39306:
        val = 4
    else:
        val = 5
    return val
Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)

cust=pd.merge(Cust_freq_count,Cust_monetary[["customerID","Monetary_Flag"]],on=["customerID"],how="left")
data=pd.merge(data,cust,on=["customerID"],how="left")
df1=data[["customerID","store_code","Monetary_Flag","Freq_Flag","Recency_Flag_y"]]

In [ ]:

Cust_freq=data[["date","customerID"]].drop_duplicates()

Cust_freq_count=Cust_freq.groupby(["customerID"])["date"].count().reset_index().sort_values("date", ascending=False)


def f(row):
    if row["date"] <= 5:
        val = 3
    elif row["date"] >=5 and row["date"] <= 10:
        val = 2
    else :
        val = 1
    return val
Cust_freq_count["Freq_Flag"] = Cust_freq_count.apply(f, axis=1)
Cust_freq_count.drop(["date"],axis=1,inplace=True)


Cust_recn=data[["date","customerID"]].drop_duplicates()
def f(row):
    if row["date"] >= 201612:
        val = 1
    elif row["date"] < 201612 and row["date"] > 201604:
        val = 2
    elif row["date"] <= 201604 and row["date"] > 201512:  
        val=3
    else: 
        val = 4
    return val

Cust_recn["Recency_Flag"] = Cust_recn.apply(f, axis=1)
data_freq = Cust_recn.groupby("customerID",as_index=False)["Recency_Flag"].max()



Cust_monetary = data.groupby(["customerID"])["payment_amount_by_tender"].sum().reset_index().sort_values("payment_amount_by_tender", ascending=False)
unique_price=Cust_monetary[["payment_amount_by_tender"]].drop_duplicates()
unique_price=unique_price[unique_price["payment_amount_by_tender"] > 0]
unique_price["monetary_Band"] = pd.qcut(unique_price["payment_amount_by_tender"], 5)
unique_price=unique_price[["monetary_Band"]].drop_duplicates()


def f(row):
    if row["payment_amount_by_tender"] <= 11000:
        val = 3
    elif row["payment_amount_by_tender"] > 11000 and row["payment_amount_by_tender"] <= 23000:
        val = 2
    else:
        val = 1
    return val
Cust_monetary["Monetary_Flag"] = Cust_monetary.apply(f, axis=1)
Cust_monetary.drop(["payment_amount_by_tender"],axis=1,inplace=True)

data_freq=data_freq.merge(Cust_monetary,on=["customerID"],how='left')
train=data_freq.merge(Cust_freq_count,on=["customerID"],how='left')

In [ ]:
prods["store_code"].unique()

In [ ]:
df1=df1.drop_duplicates()
df2=df2.drop_duplicates()
df3=df3.drop_duplicates()
df4=df4.drop_duplicates()
df5=df5.drop_duplicates()
df6=df6.drop_duplicates()
df7=df7.drop_duplicates()

In [ ]:
[['customerID','store_code','Gender','DOB']]

In [ ]:
check1=prods[prods["store_code"]==4986][['customerID','store_code']]
check2=prods[prods["store_code"]==2906][['customerID','store_code']]
check3=prods[prods["store_code"]==2655][['customerID','store_code']]
check4=prods[prods["store_code"]==2615][['customerID','store_code']]
check5=prods[prods["store_code"]==3692][['customerID','store_code']]
check6=prods[prods["store_code"]==4843][['customerID','store_code']]
check7=prods[prods["store_code"]==4796][['customerID','store_code']]

In [ ]:
test1=pd.merge(check1,df1,on=['customerID','store_code'],how='left')
test2=pd.merge(check2,df2,on=['customerID','store_code'],how='left')
test3=pd.merge(check3,df3,on=['customerID','store_code'],how='left')
test4=pd.merge(check4,df4,on=['customerID','store_code'],how='left')
test5=pd.merge(check5,df5,on=['customerID','store_code'],how='left')
test6=pd.merge(check6,df6,on=['customerID','store_code'],how='left')
test7=pd.merge(check7,df7,on=['customerID','store_code'],how='left')


In [ ]:
test=pd.DataFrame()

In [ ]:
test=test.append(test1)
test=test.append(test2)
test=test.append(test3)
test=test.append(test4)
test=test.append(test5)
test=test.append(test6)
test=test.append(test7)

In [ ]:
test=test.fillna(0)

In [ ]:
data=tender.copy(deep=True)

In [ ]:
df=prods[["customerID","store_code"]]

In [ ]:
train=df.merge(train,on=["customerID"],how='left')

In [ ]:
train=train.fillna(0)

In [ ]:
# Now cluster the customers into 34 clusters

cld=train.drop(['store_code','customerID'], axis=1)
rbs = RobustScaler()
cld2 = rbs.fit_transform(cld)
    
km1 = KMeans(n_clusters=34)
km2 = km1.fit(cld2)
label = km2.predict(cld2)
silhouette_score(cld2, label)

In [ ]:
customers=train["customerID"]
store=train["store_code"]

In [ ]:
sub1 = pd.DataFrame({'customerID':customers, 'store_code':store, 'cluster':label},)

In [ ]:
cols=['customerID','store_code','cluster']

In [ ]:
sub1=sub1[cols]

In [ ]:
np.savetxt('/Users/shashank/Desktop/sub/subOne_5.txt', cld2)
sub1.to_csv('/Users/shashank/Desktop/sub/subtwo_5.csv', index=False)